In [1]:
# ==========================================
# File: project_cdft/notebooks/02_model_tuning_xgb.ipynb
# ==========================================
# [CDFT 专属] 模型调优 (专注版)
# 优化策略:
# 1. 减少搜索维度: 固定 colsample_bytree=1.0 (特征少，不随机丢弃)
# 2. 减少补偿效应: 只搜 L2 正则 (lambda)，固定 L1 (alpha)=0
# 3. 聚焦树结构: 仅在 深度(Depth) 和 学习率(Eta) 上做文章

import os
import json
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV, RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from scipy.stats import randint, uniform, loguniform

# 1. 环境准备
if 'notebooks' in os.getcwd(): os.chdir('..')
print(f"🚀 [02] 启动 CDFT 调参 (Focus Mode) | 目录: {os.getcwd()}")

# 2. 读取数据 (只读 Train)
try:
    train = pd.read_csv('data/processed/train.csv')
    print(f"✅ 读取训练集: {train.shape}")
except FileNotFoundError:
    raise FileNotFoundError("❌ 找不到 data/processed/train.csv，请先运行 01 号文件")

target_col = 'Target_Log1o2' if 'Target_Log1o2' in train.columns else 'Target'
y_train = train[target_col]
X_train = train.drop(columns=[target_col, 'SMILES_Meta'], errors='ignore').select_dtypes(include=[np.number])

# 3. 构建管道 (CDFT 数值通常需要标准化)
# 注意: 这里去掉了 PowerTransformer，因为 CDFT 物理量通常不需要强行正态化，StandardScaler 足够
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', XGBRegressor(objective='reg:squarederror', n_jobs=-1, random_state=42))
])

# 4. 参数空间 (瘦身版)
# 去掉了 colsample_bytree (固定1.0)
# 去掉了 reg_alpha (固定0)
param_dist = {
    # 核心复杂度控制
    'xgb__max_depth': [2, 3, 4],  # 极简树深，防止过拟合

    # 学习率与树数量 (这对冤家还是要搜一下的)
    'xgb__learning_rate': loguniform(0.01, 0.15), # 稍微限制上限，不让它学太快
    'xgb__n_estimators': randint(80, 300),        # 也不需要上千棵树

    # 稳定性控制
    'xgb__subsample': uniform(0.6, 0.3),          # 0.6 ~ 0.9，保证样本覆盖度

    # 正则化 (只搜 L2)
    'xgb__reg_lambda': loguniform(0.1, 5.0),      # L2 正则
    'xgb__reg_alpha': [0],                        # L1 固定为 0，减少维度干扰

    # 特征采样 (固定)
    'xgb__colsample_bytree': [1.0]                # CDFT 特征都是精华，全都要！
}

# 5. 执行搜索
print(">>> [Step 2] Tuning XGBoost (5-Fold CV)...")
# 这里的 n_iter 可以稍微设小一点，因为搜索空间变小了
rkf = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
search = RandomizedSearchCV(
    pipeline,
    param_dist,
    n_iter=60,          # 60次足矣，空间不大
    cv=rkf,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

search.fit(X_train, y_train)

# 6. 保存结果
best_params = search.best_params_
best_rmse = np.sqrt(-search.best_score_)

print(f"\n🏆 最佳参数: {best_params}")
print(f"   最佳 CV RMSE: {best_rmse:.4f}")

os.makedirs('results/models', exist_ok=True)
# 注意：这里存的时候要把 pipeline 的前缀 'xgb__' 去掉，方便 03 号文件直接用
clean_params = {k.replace('xgb__', ''): v for k, v in best_params.items()}

with open('results/models/best_params_xgb.json', 'w') as f:
    json.dump(clean_params, f, indent=4)

print(f"💾 参数已保存至 results/models/best_params_xgb.json")

🚀 [02] 启动 CDFT 调参 (Focus Mode) | 目录: D:\HO\my_smiles_project\y_g.baby\project_cdft
✅ 读取训练集: (105, 25)
>>> [Step 2] Tuning XGBoost (5-Fold CV)...
Fitting 25 folds for each of 60 candidates, totalling 1500 fits



🏆 最佳参数: {'xgb__colsample_bytree': 1.0, 'xgb__learning_rate': np.float64(0.021371153508685993), 'xgb__max_depth': 2, 'xgb__n_estimators': 86, 'xgb__reg_alpha': 0, 'xgb__reg_lambda': np.float64(0.4103449926306416), 'xgb__subsample': np.float64(0.8955619904286328)}
   最佳 CV RMSE: 1.2722
💾 参数已保存至 results/models/best_params_xgb.json
